In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from datetime import datetime
from dateutil import relativedelta


## 确定目标

某航空公司面临着旅客流失、竞争力下降和航空资源未充分利用等经营危机。

希望建立合理的客户价值评估模型，对客户进行分群。分析比较不同客户群的客户价值，并制定相应的营销策略。




## 明确需求


核心需求：
1. 分析比较不同客户群的客户价值
2. 并制定相应的营销策略

本节课重点讲需求1，如何对客户进行分类并分析价值。需求2是需要行业经验积累的，在此略去。

**案例数据概述**：从航空公司系统内的客户基本信息、乘机信息以及积分信息等详细数据中，根据末次飞行日期（LAST_FLIGHT_DATE），抽取20120401至20140331内所有乘客的详细数据，总共有62988条记录。其中包含了会员卡号、入会时间、性别、年龄、会员卡级别、工作地城市、工作地所在省份、工作地所在国家、观测窗口结束时间、观测窗口乘机积分、飞行公里数、飞行次数、飞行时间、乘机时间间隔和平均折扣率等44个属性。

详细字段含义：
* 会员卡号	MEMBER_NO
* 入会时间	FFP_DATE
* 第一次飞行日期	FIRST_FLIGHT_DATE
* 性别	GENDER
* 会员卡级别	FFP_TIER
* 工作地城市	WORK_CITY
* 工作地所在省份	WORK_PROVINCE
* 工作地所在国家	WORK_COUNTRY
* 年龄	age
* 观测窗口的结束时间	LOAD_TIME
* 飞行次数	FLIGHT_COUNT
* 观测窗口总基本积分	BP_SUM
* 第一年精英资格积分	EP_SUM_YR_1
* 第二年精英资格积分	EP_SUM_YR_2
* 第一年总票价	SUM_YR_1
* 第二年总票价	SUM_YR_2
* 观测窗口总飞行公里数	SEG_KM_SUM
* 观测窗口总加权飞行公里数（Σ舱位折扣×航段距离）	WEIGHTED_SEG_KM
* 末次飞行日期	LAST_FLIGHT_DATE
* 观测窗口季度平均飞行次数	AVG_FLIGHT_COUNT
* 观测窗口季度平均基本积分累积	AVG_BP_SUM
* 观察窗口内第一次乘机时间至MAX（观察窗口始端，入会时间）时长	BEGIN_TO_FIRST
* 最后一次乘机时间至观察窗口末端时长	LAST_TO_END
* 平均乘机时间间隔	AVG_INTERVAL
* 观察窗口内最大乘机间隔	MAX_INTERVAL
* 观测窗口中第1年其他积分（合作伙伴、促销、外航转入等）	ADD_POINTS_SUM_YR_1
* 观测窗口中第2年其他积分（合作伙伴、促销、外航转入等）	ADD_POINTS_SUM_YR_2
* 积分兑换次数	EXCHANGE_COUNT
* 平均折扣率	avg_discount
* 第1年乘机次数	P1Y_Flight_Count
* 第2年乘机次数	L1Y_Flight_Count
* 第1年里程积分	P1Y_BP_SUM
* 第2年里程积分	L1Y_BP_SUM
* 观测窗口总精英积分	EP_SUM
* 观测窗口中其他积分（合作伙伴、促销、外航转入等）	ADD_Point_SUM
* 非乘机积分总和	Eli_Add_Point_Sum
* 第2年非乘机积分总和	L1Y_ELi_Add_Points
* 总累计积分	Points_Sum
* 第2年观测窗口总累计积分	L1Y_Points_Sum
* 第2年的乘机次数比率	Ration_L1Y_Flight_Count
* 第1年的乘机次数比率	Ration_P1Y_Flight_Count
* 第1年里程积分占最近两年积分比例	Ration_P1Y_BPS
* 第2年里程积分占最近两年积分比例	Ration_L1Y_BPS
* 非乘机的积分变动次数	Point_NotFlight

从需求来看，是个典型的客户价值分析场景，合适使用RFM模型进行分析。

但是，上面提供的字段并不能完全适配RFM模型，且部分字段本身也能体现客户价值。

所以首先筛选出会明显影响客户价值的变量（在这一步的操作上就很依赖于业务理解能力），在RFM的基础上进行拓展，加上会员入会时间L，然后消费金额M用飞行历程和和折扣费率C代替，得到LRFMC模型。


| 字段 | 含义 |
| --- | --- |
| L | 会员入会时间距离观测窗口结束的月份数 |
| R | 客户最近一次乘坐飞机距离观测窗口结束的时间的月份数 |
| F | 客户在观测窗口内乘公司飞机的次数 |
| M | 客户在观测窗口内的累计飞行里程 |
| C | 客户在观测窗口内乘坐舱位所对应的折扣系数的平均值 |

如果按照RFM模型对客户进行分类，由于上面有5个变量，总计有2^5共计32个客户群体，划分过细，难以操作。
考虑到这实际上是一个分类问题，因此我们可以用机器学习中的聚类算法来解决。这里简单起见，采用K-means聚类算法。


In [2]:
## 原始数据
df_origin = pd.read_csv('data/air_data.csv')
df_origin

,MEMBER_NO,FFP_DATE,FIRST_FLIGHT_DATE,GENDER,FFP_TIER,WORK_CITY,WORK_PROVINCE,WORK_COUNTRY,AGE,LOAD_TIME,FLIGHT_COUNT,BP_SUM,EP_SUM_YR_1,EP_SUM_YR_2,SUM_YR_1,SUM_YR_2,SEG_KM_SUM,WEIGHTED_SEG_KM,LAST_FLIGHT_DATE,AVG_FLIGHT_COUNT,AVG_BP_SUM,BEGIN_TO_FIRST,LAST_TO_END,AVG_INTERVAL,MAX_INTERVAL,ADD_POINTS_SUM_YR_1,ADD_POINTS_SUM_YR_2,EXCHANGE_COUNT,avg_discount,P1Y_Flight_Count,L1Y_Flight_Count,P1Y_BP_SUM,L1Y_BP_SUM,EP_SUM,ADD_Point_SUM,Eli_Add_Point_Sum,L1Y_ELi_Add_Points,Points_Sum,L1Y_Points_Sum,Ration_L1Y_Flight_Count,Ration_P1Y_Flight_Count,Ration_P1Y_BPS,Ration_L1Y_BPS,Point_NotFlight
0,54993,2006/11/02,2008/12/24,男,6,.,北京,CN,31.0,2014/03/31,210,505308,0,74460,239560.0,234188.0,580717,558440.14,2014/03/31,26.250,63163.500,2,1,3.483254,18,3352,36640,34,0.961639,103,107,246197,259111,74460,39992,114452,111100,619760,370211,0.509524,0.490476,0.487221,0.512777,50
1,28065,2007/02/19,2007/08/03,男,6,NaN,北京,CN,42.0,2014/03/31,140,362480,0,41288,171483.0,167434.0,293678,367777.20,2014/03/25,17.500,45310.000,2,7,5.194245,17,0,12000,29,1.252314,68,72,177358,185122,41288,12000,53288,53288,415768,238410,0.514286,0.485714,0.489289,0.510708,33
2,55106,2007/02/01,2007/08/30,男,6,.,北京,CN,40.0,2014/03/31,135,351159,0,39711,163618.0,164982.0,283712,355966.50,2014/03/21,16.875,43894.875,10,11,5.298507,18,3491,12000,20,1.254676,65,70,169072,182087,39711,15491,55202,51711,406361,233798,0.518519,0.481481,0.481467,0.518530,26
3,21189,2008/08/22,2008/08/23,男,5,Los Angeles,CA,US,64.0,2014/03/31,23,337314,0,34890,116350.0,125500.0,281336,306900.88,2013/12/26,2.875,42164.250,21,97,27.863636,73,0,0,11,1.090870,13,10,186104,151210,34890,0,34890,34890,372204,186100,0.434783,0.565217,0.551722,0.448275,12
4,39546,2009/04/10,2009/04/15,男,6,贵阳,贵州,CN,48.0,2014/03/31,152,273844,0,42265,124560.0,130702.0,309928,300834.06,2014/03/27,19.000,34230.500,3,5,4.788079,47,0,22704,27,0.970658,71,81,128448,145396,42265,22704,64969,64969,338813,210365,0.532895,0.467105,0.469054,0.530943,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62983,18375,2011/05/20,2013/06/05,女,4,广州,广东,CN,25.0,2014/03/31,2,0,0,0,0.0,0.0,1134,0.00,2013/06/09,0.250,0.000,430,297,4.000000,4,195,12123,1,0.000000,0,2,0,0,0,12318,12318,12123,12318,12123,1.000000,0.000000,0.000000,0.000000,22
62984,36041,2010/03/08,2013/09/14,男,4,佛山,广东,CN,38.0,2014/03/31,4,0,0,0,0.0,0.0,8016,0.00,2014/01/03,0.500,0.000,531,89,37.000000,60,50466,56506,14,0.000000,0,4,0,0,0,106972,106972,56506,106972,56506,1.000000,0.000000,0.000000,0.000000,43
62985,45690,2006/03/30,2006/12/02,女,4,广州,广东,CN,43.0,2014/03/31,2,0,0,0,0.0,0.0,2594,0.00,2014/03/03,0.250,0.000,536,29,166.000000,166,0,0,0,0.000000,0,2,0,0,0,0,0,0,0,0,1.000000,0.000000,0.000000,0.000000,0
62986,61027,2013/02/06,2013/02/14,女,4,广州,广东,CN,36.0,2014/03/31,2,0,0,0,0.0,0.0,3934,0.00,2013/02/26,0.400,0.000,8,400,12.000000,12,0,0,0,0.000000,2,0,0,0,0,0,0,0,0,0,0.000000,1.000000,0.000000,0.000000,0


## 探索探索

看看数据是否能满足分析需求，统计特征如何，是否有脏数据（缺失值等）

In [3]:
def explore(df):
    df_ex = df.describe(percentiles=[], include='all').T
    df_ex['null_count'] = len(df) - df_ex['count']
#     df_ex = df_ex[['null_count', 'max', 'min']]
    return df_ex
df_explore = explore(df_origin)
df_explore

,count,unique,top,freq,mean,std,min,50%,max,null_count
MEMBER_NO,62988,NaN,NaN,NaN,31494.5,18183.2,1,31494.5,62988,0
FFP_DATE,62988,3068,2011/01/13,184,NaN,NaN,NaN,NaN,NaN,0
FIRST_FLIGHT_DATE,62988,3406,2013/02/16,96,NaN,NaN,NaN,NaN,NaN,0
GENDER,62985,2,男,48134,NaN,NaN,NaN,NaN,NaN,3
FFP_TIER,62988,NaN,NaN,NaN,4.10216,0.373856,4,4,6,0
WORK_CITY,60719,3310,广州,9385,NaN,NaN,NaN,NaN,NaN,2269
WORK_PROVINCE,59740,1185,广东,17507,NaN,NaN,NaN,NaN,NaN,3248
WORK_COUNTRY,62962,118,CN,57748,NaN,NaN,NaN,NaN,NaN,26
AGE,62568,NaN,NaN,NaN,42.4763,9.88591,6,41,110,420
LOAD_TIME,62988,1,2014/03/31,62988,NaN,NaN,NaN,NaN,NaN,0


从统计结果来看，存在票价为空值，票价最小值为0、折扣率最小值为0、总飞行公里数大于0的记录。
票价为空值的数据可能是客户不存在乘机记录造成，其他的数据可能是客户乘坐0折机票或者积分兑换产生的。如果是前者，就是错误的数据，应当在数据处理阶段予以排除



## 加工数据

将数据进行清洗，去掉脏数据。并依据之前的分析需求，将数据转换成K-means聚类算法所需要的形式：计算出所需要的五个字段并进行标准化

步骤如下：清洗脏数据 -> 加工得到五个计算字段 -> 标准化

其中五个字段的计算方法如下：
* L（会员入会时间距观测窗口结束的月数）＝LOAD_TIME－FFP_DATE = 观测窗口的结束时间－入会时间；【单位：月】
* R（客户最近一次乘坐公司飞机距观测窗口结束的月数）＝LAST_TO_END = 最后一次乘机时间至观察窗口末端时长；【单位：月】
* F（客户在观测窗口内乘坐公司飞机的次数）＝ FLIGHT_COUNT ＝ 观测窗口的飞行次数；【单位：次】
* M（客户在观测时间内在公司累计的飞行里程）＝ SEG_KM_SUM ＝ 观测窗口的总飞行公里数【单位：公里】
* C（客户在观测时间内乘坐舱位所对应的折扣系数的平均值）＝ AVG_DISCOUNT ＝ 平均折扣率【单位：无】


## 分析建模

将前面的数据进行处理，得到聚类分析的结果

In [11]:
def clean(df):
    '''清洗数据
    '''
    # 去掉票价为空的数据
    df_clean = df[df['SUM_YR_1'].notnull() & df['SUM_YR_2'].notnull()]
    # 只保留票价非零的，或者平均折扣率与总飞行公里数同时为0的记录。
    c1 = df_clean['SUM_YR_1'] != 0
    c2 = df_clean['SUM_YR_2'] != 0
    c3 = (df_clean['SEG_KM_SUM'] == 0) & (df_clean[f'avg_discount'] == 0)
    df_clean = df_clean[c1 | c2 | c3]
    return df_clean


def lrfmc(df):
    def _get_L(row):
        date1 = datetime.strptime(row['FFP_DATE'], '%Y/%m/%d')
        date2 = datetime.strptime(row['LOAD_TIME'], '%Y/%m/%d')
        r = relativedelta.relativedelta(date2, date1)
        return r.years*12 + r.months + r.days/30.0
    df['L'] = df.apply(_get_L,axis=1)
    df['R'] = df['LAST_TO_END']
    df['F'] = df['FLIGHT_COUNT']
    df['M'] = df['SEG_KM_SUM']
    df['C'] = df['avg_discount']
    return df[['L','R','F','M','C']]

def zsore(df):
    '''对数据进行标准化'''
    df = (df - df.mean(axis=0)) / (df.std(axis=0))
    df.columns = ['Z' + i for i in df.columns]
    return df
    

df_clean = clean(df_origin)
df_lrfmc = lrfmc(df_clean)
df_zsore = zsore(df_lrfmc)
df_zsore

,ZL,ZR,ZF,ZM,ZC
0,1.436590,-0.944948,14.034016,26.761154,1.295540
1,1.308539,-0.911894,9.073213,13.126864,2.868176
2,1.330081,-0.889859,8.718869,12.653481,2.880950
3,0.658713,-0.416098,0.781585,12.540622,1.994714
4,0.385857,-0.922912,9.923636,13.898736,1.344335
...,...,...,...,...,...
62974,2.075646,-0.460169,-0.706656,-0.805297,-0.065898
62975,0.558187,-0.283886,-0.706656,-0.805297,-0.282309
62976,-0.150280,-0.735611,-0.706656,-0.772332,-2.689885
62977,-1.205800,1.605649,-0.706656,-0.779837,-2.554628


In [24]:
def kmeans(df):
    '''返回聚类中心和新的带标签的数据'''
    k = 5
    kmodel = KMeans(n_clusters=k)
    kmodel.fit(df)
    
    df_center = pd.DataFrame(kmodel.cluster_centers_,columns=['L',"R","F","M","C"])
    df_center['label'] = [i for i in range(len(kmodel.cluster_centers_))]

#     print(kmodel.cluster_centers_)
    df_new = df.copy()
    df_new['label'] = kmodel.labels_
    
    
    return df_center,df_new

df_center, df_label = kmeans(df_zsore)
label_count = df_label.groupby("label").apply(len)
df_center['count'] = label_count
df_center

,L,R,F,M,C,label,count
0,0.481018,-0.798320,2.485689,2.426319,0.281230,0,5310
1,-0.697267,-0.408659,-0.167531,-0.169015,-0.212456,1,25355
2,1.151367,-0.370073,-0.093982,-0.103110,-0.128386,2,16094
3,-0.314004,1.669323,-0.573230,-0.537636,-0.132951,3,12440
4,0.169446,-0.079696,-0.104962,-0.084930,2.692050,4,2845


分析结果并给出解决方案：哪些是高价值的客户群体，每个群体应该采用怎样的营销策略。

* L（会员入会时间距观测窗口结束的月数）＝LOAD_TIME－FFP_DATE = 观测窗口的结束时间－入会时间；【单位：月】
* R（客户最近一次乘坐公司飞机距观测窗口结束的月数）＝LAST_TO_END = 最后一次乘机时间至观察窗口末端时长；【单位：月】
* F（客户在观测窗口内乘坐公司飞机的次数）＝ FLIGHT_COUNT ＝ 观测窗口的飞行次数；【单位：次】
* M（客户在观测时间内在公司累计的飞行里程）＝ SEG_KM_SUM ＝ 观测窗口的总飞行公里数【单位：公里】
* C（客户在观测时间内乘坐舱位所对应的折扣系数的平均值）＝ AVG_DISCOUNT ＝ 平均折扣率【单位：无】

根据上面五个指标的定义，L，F,M,C越高，R越低的客户，价值越高。

五个群体的分析如下：

* 0（重要保持客户）:L中等，R低，F高，M、C高，意味着这些客户经常乘坐公司的航班，里程长，是忠实的老客户。应当好好维系关系
* 1（重要发展客户）:L低，R低，F低，M、C中等，意味着这些是新客户，最近有乘坐公司的航班，未来很有价值，应该加大促销力度，让他们转化为忠实的老客户。
* 2（重要挽留客户）:L高，R中等，F中等，M、C中等，意味着这些是老用户，但最近没有乘坐公司的航班，需要及时挽留，提升客户满意度
* 3、4（低价值客户与一般客户）: L低，R高，F低，M、C低，可能是在打折促销时才乘坐航班，不必投入过多营销资源
